In [3]:
import pandas as pd
import numpy as np
from io import StringIO
import re
from scipy.optimize import fsolve
from astropy import constants as const
import emcee
import corner
import matplotlib.pyplot as plt

In [2]:
from tools import *
%load_ext autoreload
%autoreload 2

First, we will generate our data (radial velocity values) using a simulation!

The radial velocity of a star in a binary system is given by:
$$v(t) = \kappa[cos(f+\omega)+e \cos\omega + v_0$$
where
$$\kappa =\dfrac{(2\pi G)^{1/3}m\sin I}{T^{1/3}(M+m)^{2/3}\sqrt{1-e^2}}$$

$$tan(f/2)= \sqrt{\dfrac{1+e}{1-e}}\tan(u/2)$$

$$u-e\sin u = \dfrac{2\pi}{T}(t-\tau)$$

#### insert descr. of parameters!!!!!!!!! ####

For each parameter (like eccentricity) we have a range of possible values. These are as follows: 

########## insert ###########

Now, for each parameter, we'll randomly select values from a uniform distribution of their possible values.



In [65]:
def run_sim(sim_duration=3e8, num_data_points=100):
    """ (int, int) -> (pd.DataFrame)
    Arguments:
        sim_duration: (int) time elapsed during simulation
        num_data_points: (int) number of data points in simulation 
    Returns a pandas DataFrame with time and radial velocity
    """
    m = np.random.uniform(0, 1) # in solar masses
    M = np.random.uniform(0.072, 150) # in solar masses
    e = np.random.uniform(0, 1)
    I = np.random.uniform(-np.pi, np.pi)
    omega = np.random.uniform(0, np.pi/2)
    T = np.random.uniform(3282.3503, 3.46896e13) # in seconds
    tau = np.random.uniform(3282.3503, 3.46896e13) # in seconds
    v_0 = np.random.uniform(-1000, 1000) # in m/s
    
    t = np.linspace(0, sim_duration, num_data_points)
    
    radial_velocities = radial_velocity(t, m, M, T, I, e, v_0, omega, tau)
    # adding random Gaussian noise
    radial_velocities += np.random.normal(20, 10, len(t))
    
    data = {'Time':t, 'Radial Velocity':radial_velocities}
    df = pd.DataFrame(data)
    return df


In [4]:
class BinarySystem:
    """
    Represents a Binary System
    """
    def __init__(self, data=None, parameters=None, num_points=None):
        """ (self, pd.DataFrame, np.array(), int)


        If no inputs are given, then raise a ValueError. 
        If data is given, then use that data (assumes proper format).
        If parameters and num_points are given, then generates num_points radial velocity data (adds Gaussian noise).
        If only num_points is given, then generates random parameters and num_points radial velocity data (adds Gaussian noise).
        """
        if (data is None) and (parameters is None) and (num_points is None):
            raise ValueError('At least one initializing argument must be specified.')
        elif (data is None) and (parameters is None):
            # generating values for parameters
            self.m = np.random.uniform(0, 1) # in solar masses
            self.M = np.random.uniform(0.072, 150) # in solar masses
            self.e = np.random.uniform(0, 1)
            self.I = np.random.uniform(-np.pi, np.pi)
            self.omega = np.random.uniform(0, np.pi/2)
            self.T = np.random.uniform(3282.3503, 3.46896e13) # in seconds
            self.tau = np.random.uniform(3282.3503, 3.46896e13) # in seconds
            self.v_0 = np.random.uniform(-1000, 1000) # in m/s
            
            # generating radial velocity data from 
            t = np.linspace(0, 3e8, num_points)
            self.time = t
            radial_velocities = radial_velocity(t, self.m, self.M, self.T, 
                                                      self.I, self.e, self.v_0, self.omega, self.tau)
            # adding random Gaussian noise
            radial_velocities += np.random.normal(20, 10, len(t))
            self.radial_velocity = radial_velocities
            self.uncertainty = np.array([]) #how do we determine uncertainty when generating data?

            self.sampler = None
            self.samples = None
        elif (data is None):
            #generate random values from given parameters

            #MORE WORK *****
            #self.e = param[]

            t = np.linspace(0, 3e8, num_points)
            self.time = t
            radial_velocities = radial_velocity(t, self.m, self.M, self.T, 
                                                      self.I, self.e, self.v_0, self.omega, self.tau)
            # adding random Gaussian noise
            radial_velocities += np.random.normal(20, 10, len(t))
            self.radial_velocity = radial_velocities
            self.uncertainty = np.array([]) #how do we determine uncertainty when generating data?
        
            self.sampler = None
            self.samples = None
        elif (num_points is None) and (parameters is None):
            #no known parameters
            self.m = None
            self.M = None
            self.e = None
            self.I = None
            self.omega = None
            self.T = None
            self.tau = None
            self.v_0 = None
            
            #data is given 
            self.data = data #dataframe with all data
            self.time = self.data[0].to_numpy()
            self.radial_velocity = self.data[1].to_numpy()
            self.uncertainty = self.data[2].to_numpy()

            self.sampler = None
            self.samples = None
        else:
            raise ValueError('Only certain combinations of inputs are accepted when defining a BinarySystem.')
    

    def truth(self):
        """ (self) -> (np.array)
        If the parameters of the system are known, then return these. 
        """
        #MORE WORK *****
        ans = []
        return np.array(ans)
    
    @staticmethod
    def log_likelihood():
        pass

    @staticmethod
    def log_prior():
        pass

    @staticmethod
    def log_post():
        pass
    
    def initialize_mcmc(self, nwalkers, ndim=7):
        """ (self, int, int) -> ()
        Sets up the MCMC in self.sampler with nwalkers walkers and ndim dimension.
        """
        self.sampler = emcee.EnsembleSampler(nwalkers, ndim, BinarySystem.log_post, args=(self.time, self.radial_velocity, self.uncertainty))
    
    def run_mcmc(self, num_iter, nwalkers=None, ndim=7):
        """ (self, int, int, int) -> ()
        Runs the MCMC with num_iter iterations.
        If the MCMC is not initialized, sets it up with nwalkers walkers and ndim dimension.
        """
        if (self.sampler is None) and (nwalkers is None):
            raise ValueError('EnsembleSampler is not initialized, pass nwalkers as an argument or run self.initialize_mcmc.')
        elif self.sampler is None: #need to initialize the mcmc first
            self.initialize_mcmc(nwalkers, ndim)
        
        #MORE WORK *****
        initial_pos = np.array(np.random.uniform(0, 0), #mu
                                np.random.uniform(0, 0))
        
        self.sampler.run_mcmc(initial_pos, num_iter, progress=True)

    def get_samples(self, flat=False, thin=1, discard=0):
        """ (self, boolean, int, int) -> (np.array)
        Returns the chain of Monte Carlo steps.
        flat : determines whether or not to flatten the array.
        thin : take only every thin steps from the chain.
        discard : discards the first given number of steps. 
        """
        if self.sampler is None:
            raise ValueError('Must run the MCMC before accessing the samples.')
        return self.sampler.get_chain(flat=flat, thin=thin, discard=discard)
    
    def save_samples(self, flat=False, thin=1, discard=0):
        """ (self, boolean, int, int) -> ()
        Saves the chain of Monte Carlo steps to an instance attribute.
        flat : determines whether or not to flatten the array.
        thin : take only every thin steps from the chain.
        discard : discards the first given number of steps. 
        """
        if self.sampler is None:
            raise ValueError('Must run the MCMC before accessing the samples.')
        self.samples = self.sampler.get_chain(flat=flat, thin=thin, discard=discard)

    def plot_data(self, ls='', marker='.', color='blue'):
        plt.figure()

        plt.errorbar(self.time, self.data, self.uncertainty, ls=ls, marker=marker, color=color)

        plt.xlabel('Time (s)')
        plt.ylabel('Radial Velocity (m/s)')
        plt.show()

    def trace(self, ndim=7, flat=False, thin=1, discard=0):
        """ (self, int, boolean, int, int) -> ()
        Displays trace plot of the MCMC.
        """
        f, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True)
        samples = self.get_samples(flat=flat, thin=thin, discard=discard)
        labels = ["a", "b"] #CHANGE LABELS *****
        for i in range(ndim):
            ax = axes[i]
            ax.plot(samples[:, :, i], alpha=0.3)
            ax.set_xlim(0, len(samples))
            ax.set_ylabel(labels[i])
            #ax.yaxis.set_label_coords(-0.1, 0.5)
        axes[-1].set_xlabel("Step number");

    def corner(self, thin=15, discard=100, quantiles=None):
        """ (self) -> ()
        Displays corner plot of the MCMC results
        """
        flat_samples = self.get_samples(flat=True, thin=thin, discard=discard)
        if quantiles is None:
            fig = corner.corner(flat_samples, labels=labels); #CHANGE LABELS *****
        else:
            fig = corner.corner(flat_samples, labels=labels, quantiles=[0.16, 0.5, 0.84]); #CHANGE LABELS *****

Finally, we will add random Gaussian noise to each simulated radial velocity value. 

Now, we will try to recover these parameter values using an MCMC!